In [1]:
import os
import re
import sys
import glob
import math
import logging
from pathlib import Path
from pprint import pprint

import numpy as np
import scipy as sp
import sklearn

import spacy
import tika
from tika import parser

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
from wb_nlp.processing import document

In [3]:
## Hints

# nlp = spacy.load('en_core_web_sm')

This notebook contains examples of how the `PDFDoc2Txt` class can be used to convert pdf documents into formatted text. Additional methods implemented in this class can also be applied to raw texts extracted from PDFs.

We start by creating an instance of the `PDFDoc2Txt`—`pdf_parser`.

In [4]:
pdf_parser = document.PDFDoc2Txt()

# Parsing a pdf file

Parsing a pdf file starts with the `parse` method. This method accepts a buffer of byte object or a string to a url or file path. The source type must be specified for the parser to correctly execute the processing.

Below is the implementation of the `parse` method. Tika is the main driver of the parser. We use the `xmlContent` flag to specify that we want to get an xml formatted output. The xml output contains relevant structure that we can leverage to generate an informed reconstruction of the document.

In [5]:
??pdf_parser.parse

Signature: pdf_parser.parse(source: Union[bytes, str], source_type: str = 'buffer') -> str
Source:   
    def parse(self, source: Union[bytes, str], source_type: str='buffer') -> str:
        """Parse a PDF document to text from different source types.

        Args:
            source:
                Source of the PDF that needs to be converted.
                The source could be a url, a path, or a buffer/file-like object
                to the PDF file.
            source_type:
                Specification of which source type is being provided in `source`.

        Returns:
            A string containing the parsed pdf file.

        """
        if source_type == 'url':
            buf = requests.get(source)
            pdf_text = parser.from_buffer(buf.content, xmlContent=True)

        elif source_type == 'file':
            pdf_text = parser.from_file(source, xmlContent=True)

        elif source_type == 'buffer':
            pdf_text = parser.from_buffer(source, xmlContent=

## Processing a single page

The xml returned by Tika contains page information captured by div tags. We used this to process documents by page.

The `process_page` method takes a tag element corresponding to the extracted page. Page level processing is then applied such as consolidation of paragraphs in the page and fixing footnote citations. We also perform concatenation of likely fragmented paragraphs.

In [6]:
??pdf_parser.process_page

Signature: pdf_parser.process_page(page: bs4.element.Tag) -> str
Docstring: <no docstring>
Source:   
    @staticmethod
    def process_page(page: bs4.element.Tag) -> str:
        paragraphs = []

        for p in page.find_all('p'):
            paragraph = PDFDoc2Txt.consolidate_paragraph(p.text)
            paragraph = PDFDoc2Txt.normalize_footnote_citations(paragraph)
            if not paragraph:
                continue

            prev_paragraph_end = paragraphs[-1][-1] if paragraphs and paragraphs[-1] else ''

            if prev_paragraph_end and (re.search(r'[a-zA-Z\-\,]', prev_paragraph_end) or (paragraph[0].islower())):  # .isalpha():
                paragraph = paragraphs[-1] + ' ' + paragraph
                paragraphs[-1] = paragraph
            else:
                paragraphs.append(paragraph)

        # for p in paragraphs:
        #     doc = self.nlp(p)
        #     self.sentences.extend(list(doc.sents))

        paragraphs = '\n\n'.join(paragraphs)
        return 

# Paragraph consolidation algorithm

The following method `consolidate_paragraph` contains the different heuristics for identifying fragmentation of paragraphs/sentences extracted from the pdf file.

This method is a static method allowing us to use this on arbitrary text document that may contain sentence level fragmentation due to OCR or other X-to-text conversion.

In [7]:
??pdf_parser.consolidate_paragraph

Signature:
pdf_parser.consolidate_paragraph(
    text_paragraph: str,
    min_fragment_len: int = 3,
) -> str
Source:   
    @staticmethod
    def consolidate_paragraph(text_paragraph: str, min_fragment_len: int=3) -> str:
        """Consolidate a `text_paragraph` with possible multiple newlines into one logical paragraph.

        Tika provides access to extracted text by paragraph. These paragraphs, however, may contain
        multiple newlines that break the paragraph arbitrarily. This function implements some heuristics
        to recover a logical representation of the paragraph.

        Args:
            text_paragraph:
                Text corresponding to an extracted paragraph from Tika.

        Returns:
            A string corresponding to a logical paragraph.

        """
        replace_chars = {'’': "'", '“': '"', '”': '"'}
        line_seps = set([' ', '-'])
        line_groups = [[]]

        text_paragraph = text_paragraph.replace('\r', '')

        lines = []
     

# Example

### Run tika docker image first.

https://hub.docker.com/r/apache/tika

```
sudo docker pull apache/tika
sudo docker run -d -p 9998:9998 apache/tika
```

The WB Docs repository contains pdf and txt versions of documents. However, some text versions are not formatted properly.

In [20]:
import requests

url = 'http://documents1.worldbank.org/curated/en/735931527600661308/text/126663-WP-PUBLIC-P164538-Malawi-Economic-Monitor-7-Realizing-Safety-Nets-Potential.txt'
txt_original = requests.get(url).content.decode('utf-8')

pdf_url = 'http://documents1.worldbank.org/curated/en/735931527600661308/pdf/126663-WP-PUBLIC-P164538-Malawi-Economic-Monitor-7-Realizing-Safety-Nets-Potential.pdf'
txt_parsed = pdf_parser.parse(source=pdf_url, source_type='url')

# Result from direct txt version

In [36]:
print(txt_original[12850:20000])

MALAWI ECONOMIC MONITOR MAY 2018
OVERVIEW
                                                          challenges related to erratic energy and water
The Malawi Economic Monitor (MEM) provides an
                                                          supply, which had a particularly negative impact
analysis of economic and structural development
                                                          on      manufacturing.    Within  services,    the
issues in Malawi. This edition was published in May
                                                          performance of the wholesale and retail trade and
2018. It follows on from the six previous editions of
                                                          distribution sub-sectors declined as a result of
the MEM and is part of an ongoing series, with
                                                          subdued domestic demand.
future editions to follow twice per year.
                                                 

# Result from PDFDoc2Txt

In [39]:
print(txt[4])

1 « MALAWI ECONOMIC MONITOR MAY 2018

OVERVIEW The Malawi Economic Monitor (MEM) provides an analysis of economic and structural development issues in Malawi. This edition was published in May 2018. It follows on from the six previous editions of the MEM and is part of an ongoing series, with future editions to follow twice per year.

The aim of the publication is to foster better- informed policy analysis and debate regarding the key challenges that Malawi faces in its endeavor to achieve high rates of stable, inclusive and sustainable economic growth.

The MEM consists of two parts: Part 1 presents a review of recent economic developments and a macroeconomic outlook. Part 2 focuses on a special selected topic relevant to Malawi's development prospects.

In this edition, the special topic focuses on Social Safety Nets. This is a defining moment for Malawi to transform its safety net. The recently approved second Malawi National Social Support Program (MNSSP II) works towards the creat

## Observations

Using the text version from the WB Docs repository shows a fragmented structure. Columnar flows in a pdf page are literally placed side-by-side. This proves to be a challenge since inferring of sentences is not straighforward as simply replacing line breaks with spaces.

On the other hand, we can see that the `PDFDoc2Txt` has managed to recover the logical sentences in the text. It was able to properly concatenate fragmented sentences and also identified which correspond to a single column flow.